In [ ]:
import pyaudio

In [ ]:
audio = pyaudio.PyAudio()

In [ ]:
FORMAT = pyaudio.paInt16
SAMPLE_RATE = 16000
CHUNK_SIZE = 512
CHANNELS = 2

#### Important
> Actual sample rate of the audio is 16000 but using 44100 and resampling it due to 16000 sample rate output is not supported by the current hardware

In [ ]:
stream = audio.open(format=FORMAT, channels=CHANNELS, output=True, rate=44100)

In [ ]:
info = audio.get_default_output_device_info()

In [ ]:
import os
import wave
import shutil
import numpy as np

from scipy.signal import resample

In [ ]:
def play_audio(filepath):
    with wave.open(filepath, 'rb') as wf:
        audio_data = wf.readframes(CHUNK_SIZE)
        while audio_data:
            audio_array = np.frombuffer(audio_data, dtype=np.int16)
            num_samples = int(len(audio_array) * 44100 / 8000)
            resampled_audio = resample(audio_array, num_samples)
            resampled_bytes = resampled_audio.astype(np.int16).tobytes()
            stream.write(resampled_bytes)
            audio_data = wf.readframes(CHUNK_SIZE)

In [ ]:
positive_samples = os.listdir('../samples/positive')
negative_samples = os.listdir('../samples/negative')
trash_dir = os.listdir('../samples/trash')

In [ ]:
wanna_continue = input("Are you sure you want to reverify your sample details [yes/no]: ")
if wanna_continue == "yes":
    for index, sample_fp in enumerate(positive_samples):
        play_audio(os.path.join('../samples/positive', sample_fp))
        keep_it = input(f"Keep this audio {index}_{sample_fp} [yes/no]: ")
        if keep_it != 'no':
            continue
        if os.path.exists(os.path.join('../samples/positive', sample_fp)):
            shutil.move(os.path.join('../samples/positive', sample_fp), os.path.join('../samples/trash', sample_fp))
        else:
            print(f'Failed to locate filepath {sample_fp} in positive samples.')


In [ ]:
# onDestroy
stream.stop_stream()
stream.close()
audio.terminate()